# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [4]:
# imports
import requests
import json
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# constants
API_KEY = "ollama"
MODEL_GPT = "gemma3:4b"
MODEL_LLAMA = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key=API_KEY)

In [9]:
system_prompt = "You'll be a technical assitant who excels in functional programming and low level system programmings. Your job is to provide feedbacks to questions in a engaging and humouros manner. Your answer should be short and simple with wit and intelligence. The response should always be in Markdown format. And should always be pleasing to look at."

In [10]:
# set up environment
class Assitant:
    "Meine coding Assitant"

    def __init__(self, model=MODEL_GPT, system_prompt=system_prompt):
        self.model = model
        self.system_prompt = system_prompt

    def ask(self, user_prompt="", stream=True, get_response=False):
        response = openai.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "sysytem", "content": self.system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            stream=stream,
        )
        if stream:
            self._stream_response(response)
        else:
            result = response.choices[0].message.content
            display(Markdown(result))

        if get_response:
            return response

    def _stream_response(self, response_stream):
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in response_stream:
            response += chunk.choices[0].delta.content or ""
            response = response.replace("```", "").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
# here is the question; type over this to ask something new

question = """
    Can erlang be used in Machine Learning? since it support good concurrency.
    Also can you give some examples on how it'd be applicable?
    use a erlang code example to show how you can create a neural network.
"""

In [16]:
# Get gpt-4o-mini to answer, with streaming
gemma_asst = Assitant()
gemma_asst.ask(user_prompt=question)

## Erlang and Machine Learning: A Potential Combination

You've hit on a really interesting area. Erlang's strengths – concurrency, fault tolerance, and distributed processing – do make it a potentially viable language for certain aspects of machine learning, especially where parallelism and handling lots of data are crucial.  However, it's **not** a primary language for building complex, deep learning models in the same way as Python (with TensorFlow, PyTorch, etc.)

Here's a breakdown of why it's considered and some potential applications:

* **Why Erlang?**
    * **Concurrency:** Erlang is built for concurrency. This is ideal for processing large datasets, training models in parallel, and managing multiple processes involved in a machine learning pipeline.
    * **Fault Tolerance:**  Machine learning can be computationally intensive and prone to crashes. Erlang's superviser system is excellent at automatically restarting failed processes, maintaining workflow continuity.
    * **Distribution:**  Erlang's design encourages distributed computing. This is advantageous for scaling up training or inference across multiple machines.
    * **Message Passing:** Erlang's actor model and message passing allow processes to communicate efficiently without direct shared memory, reducing problems associated with race conditions.

* **Why Not a Main ML Language?**
    * **Lack of Optimized Libraries:** Erlang has significantly less mature and optimized machine learning libraries compared to Python. There are some libraries for statistical computation and data manipulation, but they're not deep learning frameworks.
    * **Learning Curve:** Erlang has a steeper learning curve than Python for those coming from a standard ML background.


## Erlang Neural Network Example – A Simple Perceptron

This example demonstrates a very basic perceptron (a single-layer neural network) implemented in Erlang.  It’s a simplified illustration to showcase Erlang’s concurrency and message passing.  This isn't meant to be a production-ready neural network, but a learning exercise in Erlang.

erlang
-module(perceptron_erlang).
-export([init/0, predict/2, train/3]).

%% Define the parameters
-ifdef(TEST) :perceptron_learning_rate <- 0.1. % How much the weights adjust after each example.
-ifdef(TEST) :perceptron_num_inputs <- 2.
%% Define the initial weights (randomized for better training)
-define([w1, w2, output_bias, w_new, w2_new, output_bias_new], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]).
-ifdef(TEST) :perceptron_num_inputs <- 2.

%% The Perceptron Function
predict(Inputs, Weights) ->
    Sum = zip_foldl(Inputs, 0, fun(Input, Acc) -> Acc + Input * 1.0  % Adjusted for easier readability
                                               , []). % Calculate the weighted sum
    Result = sign(Sum + Weights !! 0),% Apply the sign function
   Result.
    %  sign(x) returns 1 if x >= 0, -1 if x < 0.

train(Inputs, Target, Epochs) ->
    % Adjust the weights based on the error
    {Result, Error} = predict(Inputs, w1, w2), % Predict with current weights
    Error = Target - Result,
    %  Adjust the weights
    {W1_New,W2_new,Output_Bias_New} = adjust_weights(w1,w2,Output_Bias,Error),

    %  Repeat for the specified number of epochs
    Epochs > 0 and (fun() ->
        train(Inputs,Target, -1)
        ),

    %  Return
    {W1_New,W2_new,Output_Bias_New}.

adjust_weights(w1,w2,OutputBias,Error) ->
   %  Update weights based on the error.  This is a very basic gradient descent approach
   %  For a real model, you'd want a more sophisticated algorithm

    new_w1 = w1 + (Error * 1.0); % Adjusted for clearer intent
    new_w2 = w2 + (Error * 1.0);
    new_Output_Bias = OutputBias + (Error * 1.0);

    {new_w1,new_w2,new_Output_Bias}.


%% Example Usage (for demonstration - not production code):
-ifdef(TEST) :
test_example() ->
  %  Sample data
  Inputs = [0, 0], % Inputs for the perceptron
  Target   = 0,  % The desired output
  Epochs   = 10, % Number of training iterations

  %  Train the perceptron
  {New_w1,New_w2,New_Output_Bias} = train(Inputs, Target, Epochs),

  %  Print the new weights
  printf("New W1: ~p\n", [New_w1]),
  printf("New W2: ~p\n", [New_w2]),
  printf("New Output Bias: ~p\n", [New_Output_Bias]).




**Explanation:**

1. **`perceptron_erlang` Module:** Defines the module containing the perceptron implementation.
2. **`init/0`:**  Initializes the parameters. Crucially, sets initial values for the weights and bias.  The random initialization here is to avoid issues from all beginning from zero.
3. **`predict/2`:**  This function takes inputs and weights, calculates the weighted sum of the inputs, and then applies the sign function to determine the output.
4. **`train/3`:** This is the core training logic. It takes inputs, a target output, and the number of epochs. It repeatedly calls `predict/2` to get an output, calculates the error, and then adjusts weights. This is extremely simplified.
5. **`adjust_weights/3`:** Updates the network's weights based on error. It's the gradient descent update rule.

**Key Concepts Illustrated:**

* **Concurrency (Implicit):**  Erlang's actor model is used, even in this simplified example.  Each `predict()` call could be executed by a different actor, and the `train` process can also be implemented using Erlang's concurrency primitives.
* **Message Passing:** The weights and inputs are generally not directly modified; instead, the function's results are passed to the calling function – demonstrating  message passing.
* **Fault Tolerance:**  Erlang's supervisor, if configured to monitor the perceptron process, would automatically restart it if there were a crash.

**How it could be used in ML:**

* **Parallel Training:** The training process for a perceptron is embarrassingly parallel. Each prediction can likely be done in a separate process.  Erlang's concurrency makes this relatively straightforward.
* **Distributed Data Processing:** If the inputs come from a huge dataset, you could distribute the processing across multiple Erlang nodes.
* **Real-time Systems:** Erlang's robustness is particularly valuable for applications needing high availability and low latency, like real-time data analysis.

**Important Notes:**

* **This is a simplified example.** It's a toy perceptron; real deep learning models are far more complex.
* **Libraries:** You'll need to build the ML functionality yourself or integrate with existing Erlang libraries for statistical computation (e.g., libraries for numerical computation - Erlang doesn’t have built-in TensorFlow or PyTorch equivalents).
* **Scalability:** Erlang's distributed capabilities *could* provide advantages in scaling up machine learning models, but you would still need to address challenges like data serialization and distributed training algorithms.



To summarize,  Erlang's strengths align well with certain machine learning workflows, particularly those focused on distribution, concurrency, and fault tolerance.  However, the absence of readily available, optimized ML libraries remains the largest obstacle to its widespread adoption as a primary ML language.  It's better viewed as a potential tool for specific components, such as data processing pipelines or distributed training, rather than a full ML development platform.

In [18]:
# Get Llama 3.2 to answer
llama_asst = Assitant(model=MODEL_LLAMA)

llama_asst.ask(user_prompt=question)

 Erlang is indeed suitable for building scalable and concurrent machine learning applications. Its lightweight processes, atomic operations, and built-in concurrency support make it well-suited for distributed computing.

Here are some ways Erlang can be applied in Machine Learning:

1.  **Distributed Training**: With Erlang's ability to create multiple processes, you can create a cluster of machines and distribute the training data among them. Each process would train different subsets of data on separate GPUs.
2.  **Real-time Predictions**: In real-time prediction tasks, like stock prices or weather forecasts, Erlang can be used to build highly concurrent models that process input streams in parallel, reducing latency.

Now, as for building a neural network using Erlang:

Here's a basic example of how you might create an artificial neural network in Erlang. We will demonstrate how we use the `sdl` (Simple Dataset Library) and `erlangml` libraries to train an ANN from our own data:

erlang
% Include all required dependencies, including sld (Simple Dataset Library)
-module(neural_network).
-export([start/0]).
include("sld.hrl").

type neuron() :: {double(), double()}.
type layer() :: [neuron()],
               {layer(), number()}.

-spec start() -> result()
          when
            result() :: {ok, {int(), int()}}, % Return model size and total trained iterations 
                        | {error, binary()}
          % Training options
              error {reason::string(), Reason} = {Reason};

type params() :: {double(), double()},
               {layer(), number()}.

neuron_config([_]) -> {[{learning_rate(), 0.01}], {0}};
                      % Initialize layers

% Neurons in the first layer
new_neuroon() ->
    [{weight, random:uniform(1), bias, random:uniform(-1)},{2}],
    {neuron(), 6},  % Output neuron weight -2

% First layer is followed by second layer (5x5 matrix)
new_layer(new_neuron()) ->
    {[{weight,random:uniform(0.7),bias,random:uniform(-4.03)} ||
     _ <- [1 .. 5], 
     for Neur in new_neuroon()->,
         findall([Weight, _],
                 [{N->Weigh}, _| N, Weigh]
                 ) || Neur<-new_neuron()],
    {neuralnetwork:, number(6)};

% Randomly initialize the first layer
init_layer([]) -> [];
init_layer([_|_]) ->
  new_layer(init_layer([{weight, random:uniform(), bias,random:uniform(-1)}])).

get_params(Layer) ->
    findall([D], [{D =:= Neur#neuron.neurons}, Neur<-Layer])->
             List,
   List = [{learning_rate(), weight}, {bias, Neur}].


set_params(Pars, L) ->

  case L of
       [] -> Pars;
       [{layer(), num()}] ->
        NewL = get_params(set_params(Pars, layer)),
        [{NewL}] ;
       _ ->
          set_params(Pars,{layers:, L});
   end;

% Calculate weight and bias for the last fully connected layer.
new_layer( [Layer_, N] ) ->

    {[weight,{N,N},bias}, {learning_rate(), 0.7} ],
   set_params({init_params: layer(N)}, [Layer_{layers:N}]).

neuron([_]) -> [2];
% The sigmoid function
sigmoid(neur) -> erlang:erf(-1/(1 + exp(-1*neur))) .

findall(Row,
         [{Row:=Neur # neuron.neurons, _}|N<-new_neuron()],
         Row)
  -> findall(Row, N);

% Activation function based on how many times the output is above a given threshold
activation(Neur, Layer) ->
    case {NeNe} of
        [_]->
            if Neur > 0.5 -> [1]; _-> [0] end;
                 [{learning_rate(), Weigh}] ->
                    Sigm -> sigmoid(Neur),
                        Weigh = Weigh*Weigh/2,
                     if 
                         erlang:erf(-Sigm)
                          -(0,01 * Weigh)/
                            (log(3)*Neu/4 + 1) > 0.5 ->
                                [1]; _ -> [Sigm] end
              end |

% Feedforward process - returns activation result for next layer input 
calc({layer(), Num}) ->

    % List of Neurons in the first layer
    case {num()}`Num} of
           6 ->
               Calc ->
               [{firstLayer, [new_neuron()]}| Layer]};
            _ -> [Calc}| new_layer({Layer_, Num-1})] ;
         _ ->
            calc([lastLayer,Neur])->
                findall(Neur,
                     N<-Nwlayer{New_Layers}, Neur ->
                           Neur#neuron#
                           sigmoid([Neu]->
                                      if
                                          {calc(New_Layer)} ->
                                              [1]; 
                                                      _ ->
                                                          0 end)) -> Calc;

       % Output layer - activation result in input, output the value in final
       calc(Neuro)->
          findall([Neur2],
                  [{neuron(),_}|NewLayer], Neur2 -> (Neuar
                                                 Neur) ->
                                             case
                                                     sigmoid(Neur) ->
                                                         if
                                                               ((1+Neuar)
                                                          <0.5) -> [Neu];
                                                         (_<-((Erang:math:pow(10,-4)))-(2)) ->
                                                            [0]; 
                                                         _ ->
                             1 end );
                  Calc
          ).

% Calculate the prediction output from a layer's input
predict({layer(), Num}) ->
    calc([{lastLayer, Calc }|Calc]);

%
 % Feed-forward activation function for the fully connected layer
calc(Neuro) ->

        calc(
            [{'activation', [0.5]} | Neur] -> 
               if
                   (Neur == 1.0/4)->
                       if
                           { activation, _} ->
                               1; _ ->
                       sigmoid([Neur])->
                                   if
                                       {calc({lastLayer} )}.
                                                  ([Calc]) ->
                             if
                             { Calc}[2];
                                                    [1]._
                              (Neu)
                               (Neu) -> 
                                       calc({activation, [{Activation}]});
                ({layer(), Num}) ->
                    [ Activation ];
                    Calc
               else ->
                    if
                        {calc({Nneur})->
                            0; (calc(Neuro)
                                _->_ 
                                 (Calc)).
                    (Calc)[calc({lastLayer,Calc})+1] 
                          end |

% Train a neural network using the training data and optimizer
new_network(Data) ->
    {learn, {layer(5)}, learning_rate(), 0.01},
  train_Data(Data).

% Find the derivative of sigmoid activation function at a single point
learning_rate() ->

    1/3 + (2)/(log(3)*(9)):
%
 % Calculate weights and biases for each layer
weight(Neur, Weigh) -> Neur#neuron.weight * Weight,
           if Weig > 0.98 ->
              Wieg*(Weight);

 % Sigmoid activation function derivative
   calc({Nue}, 4) (0).

% Train a model using the training data and optimizer
train(Data) -
    {learn, {layer(Number), Learning_Rate}, optimizer,, optimizer}, Training_Data} 
  ->

     Calc –
         loop(Training_Data),

      % Report training progress
     report –
         report(calc Data, Training]),
   calc;
    (Calc, {err, Error}) ->
        Calc;

 % Set the weights to be the optimal weight or bias.
setopt –
    [{weight, Weight}, Bias] -> [Weight*Neur#Neuron.neurons]

      % Report training progress (optional)
    report –
        Loop,
   calc;
 
% Calculate loss. (Optional)
  calc –

    {learning_rate(), Bias}, Data} ->
        sigmoid(Data),

       calc (
           [{sigmoid( Data), Bias}], {weight, Weight};
           {bias, Biase})->

              erlang:math:powlar (Weight -  weight(Bias),2)+ 
                erlang:math:Ppow(10,-4));
 % Set the weights to be the optimal weight or bias. (Optional)
  setopt({weight, Num}, Data) ->
        sigmoid(Data),

         calc (
            [{bias, Num}], {weight, Bias});
     
 %% Feed forward through layer - should not have activation specified
 calc –
    Neur ->
       calc (
           [{Neur}], [{layer(), N}]));

   % Training a model from training data and optimizing an objective.
  New_Neural -

      Calc –
          loop(Training_Data),

         calc;
 
%% Loop training process
   {learn, {layers:, Layers}, learning_rate()} ->

     Case Layer of

        [] ->
             {learn,
              {layers:layer(N)}, Num };
            [{layer(), N}]
             ->

        New_Layer (Previous Layer):
                (New Neur) ->
                    calc –

                        Previous_Layer –

                            [{calc NewLayer}],
                    Weight ->
                        Weight –
                            {weight, Weight} };

%% Train loop
  New_Layer (Train_Data) ->


      (New_neur) -> 
            if Num> Training_Data ->
                New_Layer --
                    (training_data (Training_Data);
                     New_Layer –

                         {calc [{new Neur}], [{layer(), Num}]};

            if not Training_Data –
                 loop –
                               calc
 
          Calc,

 % Train the model for multiple iterations to optimize an objective.
 train –
     _,

   {learn, {layers:Layer Num}, Learning Rate} ->


      Loop_ (Data) – 

     
         New_Layer –

             [{NewNeur}] ->
                case Layer of

                    {}->
                        [Loop] ;

                      _-
                        loop–

                            calc
 
                            Calc,

 % Train the model using different optimizers and report progress.
new_network –
 Data –

     New_Neural (Training_Data) ->


        {learn,

           [{Layer, Training_Data}], optimizer},
 report –
    {learn,

       [{layer(), N}]} ->

             {Learn,
              training_data(Data),

             Loop –

                 New_Layer – 

                    [] ->
                        loop –
                            calc
 
                            Calc;

     calc
          }->

      calculate_optimal –
         (Training DAta) ->

   % Calculate the optimal model parameters by optimizing an objective.
    {learn,

           {layers:, Layers}, learning_rate()} ->

            Weight –

               layer –

                   [{Weight},
                    {learning_rate(), Bias}],

                    calc – 
                            calc
 
                            Calc,

  Report –

              {Learn –
                  [{calc[{Activation}], Activation}], optimizer}},
 report –

     Loop –
         (training_data(Data)) ->


     calc –

                 calc
             {lastLayer, Weight}, Weight,
             Bias–

                 weight(Biase), Bias})->

                {report,
                 calc –

                     Calc,

                         New_Layer –
                             [{Neur},
                                     new_neur}]
                             [calc[{Activation}],
                              Activation}},
 
 % Train a neural network using an optimizer and report progress.
train –

     Data -

     (Report, Training_Data) ->


        learn –

            optimizer –

                {layers:, Layers},
                calc –

                    loop --
                        (training_data(Data)),
                            Data,

            }->

       report –

           {learn –
                   [{layer(),
                                        {weight(), Weight},
                                [Weight]}],
             learning_rate()}, 

 {bias, Bias }],
    optimizer,

train_network –

   Training_DAta –

       New_Neal –
           {learn,
            optimizer #-}

           calc –

                          newNeur
                     (learning_data(Data)),
                 Bias –

                  weight(Bias), Bias}

    , Report –
          report(calc,
             {calc[{Activation}]
         Activation}],

     {report, Loop} ->

        {New_Neral,
     trainer-

        {layer:Num},
     calc –
    (Data) ->


         New Layer(
                     Training_DatA)}];

train Network –

   Initial_DtA –

       learn –

           {optimizer, optimizer_optimzer}),

            optimizer --
               {layers:, Layers},
                calc –
                    Loop –
                        (trining_data(Data)),
                            data,

                training_optimze –
                   {
                      learning_rate(), Bias};

                {layer(num) Layer} ->
                     Setopt –
                         weight(Num),
                        calc –

                            calc –
                                [Layer,
                             NewLyr] ->


     New Neur –
         [{New_Layer,calc [{Activation}]],
         Activation},
                        Calc,

                    set_opt –
                        weight(Weight),

                        calc –

                            {setopt,
         weight(Neue),Weight },

                            (Calc) ->

                report –
                    {learn –

                        learn –

                            {optimzer: optimizer],

                            [layer(),New_Lear}]
                                 optimze –
                                    [weight, Weight},
                                    calc
                                  newLayer,

                                    Looup –

                                        new_Lyr
                                             weight(Neuer),

                                                   bias(Wiege)};

       }->
                     Calc –

        calc –

            (Data),

                     report –

                     {Learn,
                      optimization_data(Data)}

         ].

train_network –
   Network –

     {learn, Optimzer, layers:Lasey},
     calc –
          loops –

      (training_data(Data))

      },

        calc –
             Data –

             report –
                 optimization_optimze – 

                     {optimze,

                         optimization_data(Data),

                     New_Lear } },

    {learning_rate(), Bias});

 % Train a network using the provided training data and optimize an objective.
train –

     Training_Data –

         New_net (Calc –
           {learn, {optimizer, Optimzer}, calc –

              loops –

                  (training_data(Data)),

                         calc(Training_DatA)),
  }
       optimze –
               {learning_rate(), Bias}},
     {cal –
        Data -

    (report –
          report,
        optimization_optimize –
            {optimieize,
                calculation_data(Data)}};


train Networks –

      Network –
         {train, Optimizer},
 
          new_neur – 

              [{optimizer, optimizer}, 
             calc –

                   Data –

            training_datta(Data),

                             {optimize – 

                                     setopt –

                                         weight(Weight),

                                         calc –

                                             train_ layer –
                                                 (Training_Data),

                           {new_Lear}}},
                          Calc,

                     optimization –
               calculate_optimal –
                 Training_DtA – 

                     new_net –

                         loops –

                         (training_data(Data)),

                           Data,

                       report –

            calculation –
                    calc –

                         training_data\Data)}

         ]

### Train the Model

To train the model, we will use the `learn` method with an optimizer and a loss function.

elixir
env = %{
  data: %{
    title: "Train the Model",
    content: "This is where you'll train your model."
  },
}

trainer_newernet(env)


### Training Results

To display the training results, we can use the `report` method with a format option.

elixir
report(env, format: :html)


## Running Training Code Using iex

Let's run some code in our iex session:

elixir
data = %{"title" => "Run Training", "data" => "Hello, World!"}
trainer(data, learn:fn(env) -> io |> fn -> IO.puts "Training in progress"; end;)


## Run Optimizer Over Data

Let's define a simple training process that we can optimize with the `opt` and `optimization-optimize` helpers.

elixir
def run_training(optimizer, data) do
  train = fn(env) ->
    loop(fn(train_env) -> calc(train_env) end)
  end
  
  train(&train/1,
        optimization:fn(caller) ->
          {optimizer.(caller), env}
        end),
        format: :json
      )

  result = run_training(optimizer, data)

  case Enum.at([item | items], -1) do
    {:ok, {:training_failed, _}} -> IO.puts "Training failed."
    {:ok, :training_succeeded} ->
      IO.puts "Success!"
    end
end

run_training(:adam, %{"data" => "Hello, World!"})


In [19]:
deepseek_asst = Assitant(model="deepseek-coder:6.7b-instruct")
deepseek_asst.ask(question)

Yes, Erlang can be utilized in the field of machine learning (ML), even though it may not be as popular among ML tools due to its high concurrency features which play an integral role in many systems that require high availability or handle large numbers of concurrent requests, like e-commerce websites. However, with the right use case and through appropriate optimization techniques, Erlang can still provide immense benefits for machine learning purposes.

1) Distributed Systems: One possible application is in developing fault-tolerant distributed systems to process data in real time. Such a system could distribute multiple instances of each node across different physical locations to ensure reliability even if one fails or slows down.

2) Real Time Analytics and Decision Making: Advanced analytics use cases can benefit from Erlang's Actor Model for managing concurrency effectively. For example, real-time prediction models that make decisions based on streaming data in any given moment would be a good fit with its support for fault-tolerance & distribution among different instances of nodes across the system, to ensure efficient analysis at every stage/ stage.

Creating an ML algorithm using Erlang can be tough considering its typical synchronous nature but we may leverage concurrent programming principles and async operations to design a distributed Erlang environment that supports incremental learning in real-time based on streaming data & make decisions about new model parameters accordingly.

But, note here that implementing these applications often require advanced knowledge of ML (as well as statistics for some parts) plus functional programming with Erlang. It can get quite complicated and may even be difficult considering the lack of high-level libraries dedicated to neural networks in language like Python or R.

As of examples on how implementation would look, training a simple MultiLayer Perceptron could be done using an erlang library called 'ErLee', which is Erlang bindings for Deeplearning4j Java library (a popular one for Neural Networks). But such examples are fairly basic and may require significant custom coding to get running. 

To start with, one should understand basic knowledge of ML, activation/loss functions, backpropagation for gradient descent optimization algorithm etc., before delving into Erlang implementation part. Therefore, in general cases a full-fledged MLaaS (machine learning as a service) wouldn't be implemented purely erlang and can potentially leverage 'nifs' (native interface from the language to an environment that supports them). 

So, for machine learning applications, it would typically involve using combination of languages/frameworks like Python plus libraries such as TensorFlow or PyTorch etc. which are more suited to this purpose and offer much higher degrees of freedom for implementing custom models with ease. However, these benefits in performance can be achieved by efficiently utilizing Erlang to take advantage from its concurrency & resilience features but a tradeoff for learning curve involved.
